# Revisiting Automatic Differentiation

In machine learning, we *train* models, updating them successively so that they get better and better as they see more and more data. Usually, *getting better* means minimizing a *loss function*, a score that answers the question "how *bad* is our model?" With neural networks, we typically choose loss functions that are differentiable with respect to our parameters.
Put simply, this means that for each of the model's parameters, we can determine how much *increasing* or *decreasing* it might affect the loss. While the calculations for taking these derivatives are straightforward, requiring only some basic calculus, for complex models, working out the updates by hand can be a pain (and often error-prone).

# 자동 미분 재방문

기계 학습에서 모델을 *훈련*하여 계속해서 업데이트하여 더 많은 데이터를 보면서 점점 더 나아지게 만듭니다. 일반적으로 *더 나아지는 것*은 *손실 함수*를 최소화하는 것을 의미하며, 이는 "우리 모델은 얼마나 *나쁜*가?"라는 질문에 답하는 점수입니다. 신경망에서 우리는 일반적으로 매개변수에 대한 미분 가능한 손실 함수를 선택합니다. 간단히 말하면, 이는 모델의 각 매개변수에 대해 손실에 얼마나 *증가* 또는 *감소*하는 영향을 결정할 수 있는 것을 의미합니다. 이러한 미분값을 계산하는 계산은 기본적인 미적분만 필요로 하지만 복잡한 모델의 경우 수동으로 업데이트를 계산하는 작업은 고통스럽고 (때로는 오류가 발생할 수 있음) 어렵습니다.

In [1]:
import torch
x = torch.randn(4, dtype=torch.float32).reshape((4, 1))
x.requires_grad=True
print(x)

tensor([[ 0.8945],
        [-0.3214],
        [-0.7774],
        [ 0.9983]], requires_grad=True)


In [2]:
y = 2*torch.mm(x.t(),x) # y = 2* ((w,x,y,z)의 전치행렬*(w,x,y,z)) => mm=행렬곱셉 => y = 2* (w^2 + x^2 + y^2 + z^2)
print(y)
y.backward()
print("x.grad:", x.grad)
print("x.grad_fn:", x.grad_fn)
print("y.grad_fn:", y.grad_fn)

tensor([[5.0086]], grad_fn=<MulBackward0>)
x.grad: tensor([[ 3.5781],
        [-1.2854],
        [-3.1094],
        [ 3.9932]])
x.grad_fn: None
y.grad_fn: <MulBackward0 object at 0x7fa58846c760>


# `Back Propagation Using Chain Rule`

*Caution: This part is tricky and not necessary to understanding subsequent sections. That said, it is needed if you want to build new layers from scratch. You can skip this on a first read.*

Sometimes when we call the backward method, e.g. `y.backward()`, where
`y` is a function of `x` we are just interested in the derivative of
`y` with respect to `x`. Mathematicians write this as
$\frac{dy(x)}{dx}$. At other times, we may be interested in the
gradient of `z` with respect to `x`, where `z` is a function of `y`,
which in turn, is a function of `x`. That is, we are interested in
$\frac{d}{dx} z(y(x))$. Recall that by the chain rule

$$\frac{d}{dx} z(y(x)) = \frac{dz(y)}{dy} \frac{dy(x)}{dx}.$$

So, when ``y`` is part of a larger function ``z`` and we want ``x.grad`` to store $\frac{dz}{dx}$, we can pass in the *head gradient* $\frac{dz}{dy}$ as an input to ``backward()``. The default argument is ``torch.ones_like(y)``. See [Wikipedia](https://en.wikipedia.org/wiki/Chain_rule) for more details.

# 체인 룰을 사용한 역전파

*주의: 이 부분은 까다롭고 후속 섹션을 이해하는 데 필요하지 않습니다. 그럼에도 불구하고 처음 읽을 때는 이해할 필요가 있습니다. 이 내용은 처음부터 새로운 레이어를 만들고자 할 때 필요합니다. 처음에는 이 부분을 건너 뛸 수 있습니다.*

가끔 우리는 역전파 메서드를 호출할 때, 예를 들어 `y.backward()`와 같이 호출할 때, `y`가 `x`의 함수인 도함수, 즉 $\frac{dy(x)}{dx}$에만 관심이 있을 수 있습니다. 다른 경우에는 `z`가 `y`의 함수이며 이는 `x`의 함수일 수 있습니다. 즉, $\frac{d}{dx} z(y(x))$에 관심이 있을 수 있습니다. 체인 룰에 따라 다음과 같이 표현할 수 있습니다.

$$\frac{d}{dx} z(y(x)) = \frac{dz(y)}{dy} \frac{dy(x)}{dx}.$$

따라서 `y`가 더 큰 함수 `z`의 일부이고 `x.grad`가 $\frac{dz}{dx}$를 저장하려고 할 때, *헤드 그래디언트*(head gradient)를 `backward()`에 입력으로 전달할 수 있습니다. 기본 인수는 `torch.ones_like(y)`입니다. 자세한 내용은 [Wikipedia](https://en.wikipedia.org/wiki/Chain_rule)를 참조하십시오.

In [16]:
x = torch.tensor([[0.],[1.],[2.],[3.]], requires_grad=True)
y = x * 2
z = y * x

head_gradient = torch.tensor([[10], [1.], [.1], [.01]])
#head_gradient = torch.tensor([[1.], [1.], [1.], [1.]])
z.backward(head_gradient)
print(x.grad)

tensor([[0.0000],
        [4.0000],
        [0.8000],
        [0.1200]])


### Computational Graph

Let’s assume we want to perform the following set of operations to get our result r:

<img src="img/comp_graph.jpeg" width=700>

$$
r=z^2(x^2+y)^2
$$

<span style="color:yellow"> $x, y, z$ are leaf variables!!

### Forward Pass (Propagation)

For example,  
If $x=1, y=2, z=4$, the final output is $r=144$.

<img src="img/forward_pass.jpeg" width=700>

### Backward Pass (Back Propagation)

To calculate gradients with regards to each of 3 variables we have to calculate partial derivatives at each node in the graph (local gradients).  


$$
\begin{align}
\frac{\partial r}{\partial w} &= \frac{\partial w^2}{\partial w} &= 2w \\
\frac{\partial w}{\partial v} &= \frac{\partial zv}{\partial v} &= z \\
\frac{\partial w}{\partial z} &= \frac{\partial zv}{\partial z} &= v \\
\frac{\partial v}{\partial u} &= \frac{\partial (u+y)}{\partial u} &= 1 \\
\frac{\partial v}{\partial y} &= \frac{\partial (u+y)}{\partial y} &= 1 \\
\frac{\partial u}{\partial x} &= \frac{\partial x^2}{\partial x} &= 2x
\end{align}
$$

$
\frac{\partial r}{\partial z} = \frac{\partial r}{\partial w}\frac{\partial w}{\partial z} = 2wv = 72
$<br><br>
$
\frac{\partial r}{\partial y} = \frac{\partial r}{\partial w}\frac{\partial w}{\partial v}\frac{\partial v}{\partial y} = 2wz\cdot 1 = 96
$<br><br>
$
\frac{\partial r}{\partial x} = \frac{\partial r}{\partial w}\frac{\partial w}{\partial v}\frac{\partial v}{\partial u}\frac{\partial u}{\partial x} = 2wz\cdot 2x = 4wz=192
$<br><br>


### 역전파 (Backward Pass)

3개의 변수에 대한 그래디언트를 계산하려면 그래프의 각 노드에서 부분 미분값, 즉 지역 그래디언트를 계산해야 합니다.

$$
\begin{align}
\frac{\partial r}{\partial w} &= \frac{\partial w^2}{\partial w} &= 2w \\
\frac{\partial w}{\partial v} &= \frac{\partial zv}{\partial v} &= z \\
\frac{\partial w}{\partial z} &= \frac{\partial zv}{\partial z} &= v \\
\frac{\partial v}{\partial u} &= \frac{\partial (u+y)}{\partial u} &= 1 \\
\frac{\partial v}{\partial y} &= \frac{\partial (u+y)}{\partial y} &= 1 \\
\frac{\partial u}{\partial x} &= \frac{\partial x^2}{\partial x} &= 2x
\end{align}
$$

$
\frac{\partial r}{\partial z} = \frac{\partial r}{\partial w}\frac{\partial w}{\partial z} = 2wv = 72
$<br><br>
$
\frac{\partial r}{\partial y} = \frac{\partial r}{\partial w}\frac{\partial w}{\partial v}\frac{\partial v}{\partial y} = 2wz\cdot 1 = 96
$<br><br>
$
\frac{\partial r}{\partial x} = \frac{\partial r}{\partial w}\frac{\partial w}{\partial v}\frac{\partial v}{\partial u}\frac{\partial u}{\partial x} = 2wz\cdot 2x = 4wz=192
$<br><br>

이렇게 하여 `r`을 각 변수에 대한 미분값을 계산할 수 있습니다.

In [4]:
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
z = torch.tensor(4.0, requires_grad=True)

# forward pass
u = x**2
v = u+y
w = z*v
r = w**2


print(f'r = {r}')
# backward pass
r.backward()
print(f'dr/dw = {w.grad} = {2*w}')
print(f'dr/dv = {v.grad}')
print(f'dr/du = {u.grad}\n')
print(f'dr/dx = {x.grad}')
print(f'dr/dy = {y.grad}')
print(f'dr/dz = {z.grad}')



r = 144.0
dr/dw = None = 24.0
dr/dv = None
dr/du = None

dr/dx = 192.0
dr/dy = 96.0
dr/dz = 72.0


/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_2247/470171788.py:15: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:491.)
  print(f'dr/dw = {w.grad} = {2*w}')
/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_2247/470171788.py:16: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the n

In [13]:
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
z = torch.tensor(4.0, requires_grad=True)

r = (z**2)+((x**2+y)**2)


print(f'r = {r}')
# backward pass
r.backward()
print(f'dr/dw = {w.grad} = {2*w}')
print(f'dr/dv = {v.grad}')
print(f'dr/du = {u.grad}\n')

print(f'dr/dx = {x.grad}')
print(f'dr/dy = {y.grad}')
print(f'dr/dz = {z.grad}')

r = 25.0
dr/dw = None = 24.0
dr/dv = None
dr/du = None

dr/dx = 12.0
dr/dy = 6.0
dr/dz = 8.0


/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_42322/4218990450.py:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:491.)
  print(f'dr/dw = {w.grad} = {2*w}')
/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_42322/4218990450.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on t

## <span style="color:yellow"> Lab : Compute gradients of $r$ w.r.t. $(x, y, z)$ at (1, 2, 3)</span>

The network output $r$ is given by:

$$
r=(x+y)^3z^2
$$

In [5]:
import torch

x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
z = torch.tensor(3.0, requires_grad=True)

# Forward pass
u = (x + y)**3
v = u * z**2
r = v

print(f'r = {r}')

# backward pass
r.backward()

print(f'dr/dw = {w.grad} = {2*w}')
print(f'dr/dv = {v.grad}')
print(f'dr/du = {u.grad}\n')

print(f'dr/dx = {x.grad}')
print(f'dr/dy = {y.grad}')
print(f'dr/dz = {z.grad}')


r = 243.0
dr/dw = None = 24.0
dr/dv = None
dr/du = None

dr/dx = 243.0
dr/dy = 243.0
dr/dz = 162.0


/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_2247/2817604518.py:17: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:491.)
  print(f'dr/dw = {w.grad} = {2*w}')
/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_2247/2817604518.py:18: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the